In [71]:
import requests
import string
from datetime import datetime

from lxml import etree
from lxml import html

from pandas import DataFrame as df

In [76]:

queryParam = {'fromDate': '2015-12-29'}
proxies = {'http': 'proxy.wellsfargo.com:8080', 'https': 'proxy.wellsfargo.com:8080'}
parser = etree.HTMLParser()

r = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do', params=queryParam, proxies=proxies)

r.url

u'http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do?fromDate=2015-12-29'

In [77]:
currentDate = dt.now()
yyyy = currentDate.year
mm = currentDate.month
dd = currentDate.day

In [82]:
flightTable = []

tree = etree.fromstring(r.content, parser=parser)
trList =tree.xpath('//table[@id=\'list\']/tr[not(@mr)]')

for tr in trList:
    if 'date' in tr.attrib:
        dateStr = tr.attrib['date'].split(',')[1]
        yyyy=int(dateStr.split('-')[0])
        mm=int(dateStr.split('-')[1])
        dd=int(dateStr.split('-')[2])
    
    scheduledTimeStr = tr.getchildren()[0].text
    schHH = int(scheduledTimeStr.split(':')[0])
    schMM = int(scheduledTimeStr.split(':')[1])
    
    scheduledTime = dt(yyyy, mm, dd, schHH, schMM)
    
    flightCode = tr.getchildren()[1].text
    
    city = tr.getchildren()[2].text
    
    airline = ''
    airlineRaw = tr.getchildren()[3].text
    for airlineStr in airlineRaw.split(','):
        if airlineStr.find('wm') == -1:
            airline = airline + ',' + airlineStr
    airline = airline[1:]
    
    gate = tr.getchildren()[4].text
    
    arriveTimeStr = tr.getchildren()[5].text
    if arriveTimeStr.find(':') == -1 :
        arriveTime = None
    elif arriveTimeStr.find('(') == -1 :
        timeStr = arriveTimeStr.split(' ')[1]
        arriveTime = datetime(yyyy, mm, dd, int(timeStr.split(':')[0]), int(timeStr.split(':')[1]))
    else :
        dateStr = arriveTimeStr.split('(')[1].split(')')[0]
        timeStr = arriveTimeStr.split(' ')[1]
        
        yyyyActual = int(dateStr.split('/')[2])
        mmActual = int(dateStr.split('/')[1])
        ddActual = int(dateStr.split('/')[0])
        
        hourActual = int(timeStr.split(':')[0])
        minActual = int(timeStr.split(':')[1])
        
        arriveTime = datetime(yyyyActual, mmActual, ddActual, hourActual, minActual)
        
    flightEntry = {'scheduledTime': scheduledTime, 
                  'flightCode': flightCode,
                  'city': city,
                  'airline': airline,
                  'gate': gate,
                  'arriveTime': arriveTime}
    
    flightTable.append(flightEntry)
    
flightData = df(flightTable)

flightData

,airline,arriveTime,city,flightCode,gate,scheduledTime
0,"國泰航空公司,芬蘭航空,美國航空,曼谷航空公司",2015-12-29 00:19:00,曼谷,"CX 702,AY 5858,AA 8894,PG 4567",B,2015-12-28 22:50:00
1,"國泰航空公司,港龍航空公司",2015-12-29 00:12:00,馬尼拉,"CX 934,KA 5934",B,2015-12-28 23:15:00
2,中華航空,2015-12-29 00:14:00,台北,CI 2903,B,2015-12-28 23:45:00
3,國泰航空公司,2015-12-29 00:34:00,新加坡,CX 636,B,2015-12-29 00:05:00
4,香港快運航空,2015-12-29 00:15:00,大阪/關西,UO 689,B,2015-12-29 00:05:00
5,"釜山航空,韓亞航空",2015-12-29 00:24:00,釜山,"BX 391,OZ 9741",B,2015-12-29 00:15:00
6,濟州航空,2015-12-29 01:22:00,首爾/仁川,7C 2107,B,2015-12-29 00:25:00
7,樂桃航空,2015-12-29 00:44:00,大阪/關西,MM 067,B,2015-12-29 00:35:00
8,"香港航空,中國東方航空,上海航空公司",2015-12-29 00:45:00,上海/浦東,"HX 231,MU 8982,FM 8231",B,2015-12-29 00:40:00
9,"港龍航空公司,中國國際航空公司,國泰航空公司",2015-12-29 00:37:00,北京,"KA 975,CA 6523,CX 5975",B,2015-12-29 00:40:00
